<a href="https://colab.research.google.com/github/R4hulR/mnistCNN-pytorch/blob/main/CNN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.nn.functional as F
import numpy
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#convert mnist image files into a tensor of 4-dimensions
transform = transforms.ToTensor()

In [3]:
train_data = datasets.MNIST(root = 'cnn_data',train = True,download = True, transform = transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 14233153.81it/s]


Extracting cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 433710.20it/s]


Extracting cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3546614.61it/s]


Extracting cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2666274.15it/s]

Extracting cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to cnn_data/MNIST/raw



In [4]:
print(len(train_data))

60000


In [5]:
test_data = datasets.MNIST(root = 'cnn_data',train = False,download = True, transform = transform)

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# train_data = train_data.to(device)
# test_data = test_data.to(device)

cpu


In [8]:
train_loader = DataLoader(train_data,batch_size=10,shuffle=True)
test_loader = DataLoader(test_data,batch_size=10,shuffle=True)

In [9]:
class CNN(nn.Module):
  def __init__(self) :
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    self.fc1 = nn.Linear(5*5*16,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
  def forward(self,x):
    #convulationl
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2)
    #Linear
    x = x.view(-1,5*5*16)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(x,dim=1)

In [10]:
torch.manual_seed(41)
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [12]:
import time
start_time = time.time()
#create variables to track things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
  trn_corr = 0
  tst_corr = 0
  for b,(X_train,y_train) in enumerate(train_loader):
    b+=1
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    predicted = torch.max(y_pred.data,1)[1]
    trn_corr += (predicted == y_train).sum()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if b%600 == 0:
      print(f'Epoch: {i} Batch: {b} Loss: {loss.item()}')
  train_losses.append(loss)
  train_correct.append(trn_corr)
  #test
  with torch.no_grad():
    for b,(X_test,y_test) in enumerate(test_loader):
      X_test = X_test.to(device)
      y_test = y_test.to(device)
      y_val = model(X_test)
      predicted = torch.max(y_val.data,1)[1]
      tst_corr += (predicted == y_test).sum()
  loss = criterion(y_val,y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)
current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes')

Epoch: 0 Batch: 600 Loss: 0.1623610556125641
Epoch: 0 Batch: 1200 Loss: 0.1502392590045929
Epoch: 0 Batch: 1800 Loss: 0.4744560718536377
Epoch: 0 Batch: 2400 Loss: 0.14238706231117249
Epoch: 0 Batch: 3000 Loss: 0.007758188061416149
Epoch: 0 Batch: 3600 Loss: 0.3836284875869751
Epoch: 0 Batch: 4200 Loss: 0.0038223876617848873
Epoch: 0 Batch: 4800 Loss: 0.0021286322735249996
Epoch: 0 Batch: 5400 Loss: 0.0569545142352581
Epoch: 0 Batch: 6000 Loss: 0.00038789428072050214
Epoch: 1 Batch: 600 Loss: 0.02950388565659523
Epoch: 1 Batch: 1200 Loss: 0.01223783753812313
Epoch: 1 Batch: 1800 Loss: 0.0017079260433092713
Epoch: 1 Batch: 2400 Loss: 0.004794587381184101
Epoch: 1 Batch: 3000 Loss: 0.00012589071411639452
Epoch: 1 Batch: 3600 Loss: 0.006774441804736853
Epoch: 1 Batch: 4200 Loss: 0.00024002441205084324
Epoch: 1 Batch: 4800 Loss: 0.011869433335959911
Epoch: 1 Batch: 5400 Loss: 0.0003403539885766804
Epoch: 1 Batch: 6000 Loss: 0.0003896451380569488
Epoch: 2 Batch: 600 Loss: 0.2102451026439666

In [13]:
train_accuracy = [(t/60000)*100 for t in train_correct]
test_accuracy = [(t/10000)*100 for t in test_correct]

# Print the accuracies for each epoch
for i in range(epochs):
  print(f"Epoch {i}: Train Accuracy: {train_accuracy[i]}, Test Accuracy: {test_accuracy[i]}")

Epoch 0: Train Accuracy: 93.76166534423828, Test Accuracy: 97.70999908447266
Epoch 1: Train Accuracy: 98.00833129882812, Test Accuracy: 98.55999755859375
Epoch 2: Train Accuracy: 98.6066665649414, Test Accuracy: 98.12999725341797
Epoch 3: Train Accuracy: 98.88500213623047, Test Accuracy: 98.38999938964844
Epoch 4: Train Accuracy: 99.10832977294922, Test Accuracy: 98.58999633789062


In [14]:
from PIL import Image
import torchvision.transforms as transforms

# Load and preprocess the image
image = Image.open('/content/num6.png').convert('L')  # Convert to grayscale
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST normalization
])
image_tensor = transform(image).unsqueeze(0).to(device)

# Pass the image to the model
with torch.no_grad():
    output = model(image_tensor)

# Get the predicted digit
predicted_digit = torch.argmax(output).item()
print(f"Predicted digit: {predicted_digit}")

Predicted digit: 6


In [15]:
torch.save(model.state_dict(), 'mnist_cnn.pt')